In [ ]:
#Keras support
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121 as DenseNet
from keras.optimizers import SGD
from keras.layers import Input, Dense
from keras.models import Model

from pandas import read_csv

from common import constants

In [ ]:
label_df = read_csv(constants.PROCESSED_DATASET_MAPPINGS['labels'])
n_classes = len(set(label_df["Id"]))

print("Number of classes: {}".format(n_classes))

In [ ]:
input_shape = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
                    rescale=1./255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    validation_split = 0.2)

train_gen = datagen.flow_from_dataframe(
                    label_df,
                    constants.PROCESSED_DATASET_MAPPINGS['train'],
                    x_col = 'Image',
                    y_col = 'Id',
                    target_size=input_shape,
                    batch_size=batch_size,
                    class_mode='categorical',
                    subset = 'training')

val_gen = datagen.flow_from_dataframe(
                    label_df,
                    constants.PROCESSED_DATASET_MAPPINGS['train'],
                    x_col = 'Image',
                    y_col = 'Id',
                    target_size=input_shape,
                    batch_size=batch_size,
                    class_mode='categorical',
                    subset='validation')

In [ ]:
inputs = Input(shape = (224, 224, 3))
X = DenseNet(weights = 'imagenet')(inputs)
X = Dense(n_classes, activation = 'softmax')(X)

model = Model(inputs = [inputs], outputs = [X])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()


In [ ]:
model.fit_generator(
    train_gen,
    steps_per_epoch=630,
    validation_data=val_gen,
    validation_steps = 150,
    epochs=50)

In [7]:
from cachetools import LRUCache

cache = LRUCache(5)
images = ['Naresh', 'Naresh', 'Hello', 'Follow']

img_objs = {}
candidate_images = set(images)
for image in set(candidate_images):
    #Get the image object for the current image from the cache.
    #Add to the dictionary, if it is not None.
    img_obj = cache.get(image)

    if img_obj:
        img_objs[image] = img_obj
        
for image in set(candidate_images):
    #Get the image object for the current image from the cache.
    #Add to the dictionary, if it is not None.
    img_obj = cache.get(image)

    if img_obj:
        img_objs[image] = img_obj
        
print(img_objs.keys())

dict_keys([])
